In [9]:
import random
from datetime import datetime
import sqlite3
import pandas as pd

travel_quotes = [
    "'The world is a book and those who do not travel read only one page.' – Saint Augustine",
    "'Travel is the only thing you buy that makes you richer.' – Anonymous",
    "'Life is short and the world is wide.' – Simon Raven",
    "'Not all those who wander are lost.' – J.R.R. Tolkien",
    "'Take only memories, leave only footprints.' – Chief Seattle",
    "'Jobs fill your pocket, adventures fill your soul.' – Jaime Lyn Beatty"
]

travellers = []

def find_travel_buddy():
    print("\n--- Find a Travel Buddy ---")
    destination = input("Enter the destination you are traveling to: ").strip().lower()

    conn = sqlite3.connect("travel_app.db")
    cursor = conn.cursor()
    query = "SELECT * FROM indian_travellers WHERE destination = ?"
    cursor.execute(query, (destination,))
    matching_travellers = cursor.fetchall()
    conn.close()

    if matching_travellers:
        print(f"\nTravellers going to {destination.capitalize()}:")
        seen = set()
        for traveller in matching_travellers:
            identity = (traveller[0], traveller[1], traveller[2], traveller[3])
            if identity not in seen:
                seen.add(identity)
                print(f"- {traveller[0]} (From {traveller[2]} to {traveller[3]})")
    else:
        print("No travellers found for that destination.")

def display_random_quote():
    print("\nTravel Inspiration")
    print(random.choice(travel_quotes))
    print("-" * 40)

def register_traveller():
    print("\n--- Register as a Traveller ---")
    name = input("Enter your name: ").strip()
    destination = input("Enter your travel destination: ").strip()

    while True:
        start_date = input("Enter your travel start date (YYYY-MM-DD): ").strip()
        end_date = input("Enter your travel end date (YYYY-MM-DD): ").strip()
        try:
            start_dt = datetime.strptime(start_date, "%Y-%m-%d").date()
            end_dt = datetime.strptime(end_date, "%Y-%m-%d").date()
            if start_dt > end_dt:
                print("Start date cannot be after end date. Please enter the dates again.")
            else:
                break
        except ValueError:
            print("Invalid date format. Please use YYYY-MM-DD.")

    traveller = {
        "name": name,
        "destination": destination.lower(),
        "start_date": start_date,
        "end_date": end_date,
        "todo_list": []
    }

    travellers.append(traveller)

    try:
        conn = sqlite3.connect("travel_app.db")
        cur = conn.cursor()

        # Prevent duplicate entry
        cur.execute("""
            SELECT * FROM indian_travellers 
            WHERE name = ? AND destination = ? AND start_date = ? AND end_date = ?
        """, (name, destination.lower(), start_date, end_date))
        exists = cur.fetchone()
        if exists:
            print("You are already registered with this trip.")
            conn.close()
            return

        cur.execute("INSERT INTO indian_travellers (name, destination, start_date, end_date, todo_list) VALUES (?, ?, ?, ?, ?)", 
                    (name, destination.lower(), start_date, end_date, ""))
        conn.commit()

        with open("indian_travellers.txt", "a") as file:
            file.write(f"{name},{destination.lower()},{start_date},{end_date}\n")

        conn.close()
        print(f"\nWelcome, {name}! You have been registered successfully.\n")

    except Exception as e:
        print(f"Error updating Indian travellers DB or file: {e}")

def update_schema():
    conn = sqlite3.connect("travel_app.db")
    cur = conn.cursor()

    try:
        cur.execute("PRAGMA table_info(indian_travellers);")
        columns = [column[1] for column in cur.fetchall()]
        if 'todo_list' not in columns:
            cur.execute("ALTER TABLE indian_travellers ADD COLUMN todo_list TEXT")
            conn.commit()
            print("Database schema updated: 'todo_list' column added.")
    except Exception as e:
        print(f"Error checking or updating schema: {e}")
    
    conn.close()

def manage_todo():
    print("\n--- Manage Your Travel To-Do List ---")
    name = input("Enter your name: ").strip()

    traveller = next((t for t in travellers if t["name"].lower() == name.lower()), None)

    if not traveller:
        conn = sqlite3.connect("travel_app.db")
        query = "SELECT * FROM indian_travellers WHERE name = ?"
        cursor = conn.cursor()
        cursor.execute(query, (name,))
        db_travelers = cursor.fetchall()

        if db_travelers:
            traveller = {
                "name": db_travelers[0][0],
                "destination": db_travelers[0][1],
                "start_date": db_travelers[0][2],
                "end_date": db_travelers[0][3],
                "todo_list": db_travelers[0][4].split(',') if db_travelers[0][4] else []
            }
            print(f"Traveller found in the database: {traveller['name']}")
        conn.close()

        if not traveller:
            print("Traveller not found. Please register first.\n")
            return

    while True:
        print(f"\n{name}'s To-Do Menu: 1-View | 2-Add | 3-Delete | 4-Back")
        option = input("Choose an option: ").strip()

        if option == "1":
            if not traveller["todo_list"]:
                print("Your to-do list is empty.")
            else:
                print("\nYour To-Do List:")
                for i, task in enumerate(traveller["todo_list"], 1):
                    print(f"{i}. {task}")
        elif option == "2":
            task = input("Enter a new task: ").strip()
            traveller["todo_list"].append(task)
            print(f"Task added: {task}")
            conn = sqlite3.connect("travel_app.db")
            cur = conn.cursor()
            cur.execute("UPDATE indian_travellers SET todo_list = ? WHERE name = ?",
                        (','.join(traveller["todo_list"]), traveller["name"]))
            conn.commit()
            conn.close()
        elif option == "3":
            if not traveller["todo_list"]:
                print("Your to-do list is empty.")
                continue
            for i, task in enumerate(traveller["todo_list"], 1):
                print(f"{i}. {task}")
            try:
                index = int(input("Enter task number to delete: ")) - 1
                removed = traveller["todo_list"].pop(index)
                print(f"Task removed: {removed}")
                conn = sqlite3.connect("travel_app.db")
                cur = conn.cursor()
                cur.execute("UPDATE indian_travellers SET todo_list = ? WHERE name = ?",
                            (','.join(traveller["todo_list"]), traveller["name"]))
                conn.commit()
                conn.close()
            except (IndexError, ValueError):
                print("Invalid task number.")
        elif option == "4":
            break
        else:
            print("Invalid choice, try again.")

def show_countdown():
    print("\n--- Travel Countdown ---")
    name = input("Enter your name: ").strip()

    traveller = next((t for t in travellers if t["name"].lower() == name.lower()), None)

    if not traveller:
        conn = sqlite3.connect("travel_app.db")
        query = "SELECT * FROM indian_travellers WHERE name = ?"
        cursor = conn.cursor()
        cursor.execute(query, (name,))
        db_travelers = cursor.fetchall()

        if db_travelers:
            traveller = {
                "name": db_travelers[0][0],
                "destination": db_travelers[0][1],
                "start_date": db_travelers[0][2],
                "end_date": db_travelers[0][3],
                "todo_list": []
            }
            print(f"Traveller found in the database: {traveller['name']}")
        conn.close()

        if not traveller:
            print("Traveller not found. Please register first.\n")
            return

    today = datetime.now().date()
    start_date = datetime.strptime(traveller["start_date"], "%Y-%m-%d").date()
    days_left = (start_date - today).days
    if days_left > 0:
        print(f"{days_left} day(s) left until your trip to {traveller['destination'].capitalize()}!")
    elif days_left == 0:
        print("Your trip starts today! Safe travels!")
    else:
        print("Your trip has already started or passed.")

def show_feedbacks():
    print("\n--- Show Feedbacks ---")
    
    try:
        with open("feedbacks.txt", "r") as file:
            feedbacks = file.readlines()

        if feedbacks:
            print("\nFeedbacks:")
            for i, feedback in enumerate(feedbacks, 1):
                print(f"{i}. {feedback.strip()}")
        else:
            print("No feedbacks available.")
    
    except FileNotFoundError:
        print("Error: 'feedbacks.txt' file not found.")

def compare_views_vs_registrations():
    print("\n--- Compare Views vs Registrations ---")
    views = update_and_get_views()

    conn = sqlite3.connect("travel_app.db")
    df = pd.read_sql_query("SELECT COUNT(*) as total_registrations FROM indian_travellers", conn)
    total_regs = df["total_registrations"].iloc[0]

    comparison = pd.DataFrame({
        "Metric": ["Total Views", "Total Registrations"],
        "Count": [views, total_regs]
    })

    print("\n--- Views vs Registrations ---")
    print(comparison)
    conn.close()

def update_and_get_views():
    views = random.randint(1000, 5000)
    print(f"Total Views: {views}")
    return views

def clean_database():
    print("\n--- Cleaning Duplicate Traveller Entries ---")
    conn = sqlite3.connect("travel_app.db")
    cur = conn.cursor()

    cur.execute("""
        DELETE FROM indian_travellers
        WHERE rowid NOT IN (
            SELECT MIN(rowid)
            FROM indian_travellers
            GROUP BY name, destination, start_date, end_date
        )
    """)
    conn.commit()
    print("Duplicates removed.")
    conn.close()

def extended_main():
    update_schema()
    # clean_database()  # <- Uncomment and run ONCE if you want to clean old duplicates

    while True:
        display_random_quote()
        print("\n--- Travel Planner App ---")
        print("1. Register Traveller")
        print("2. Find Travel Buddy")
        print("3. Manage To-Do List")
        print("4. Show Countdown")
        print("5. Show Feedbacks")
        print("6. Compare Views vs Registrations")
        print("7. Exit")
        
        choice = input("Choose an option: ").strip()

        if choice == "1":
            register_traveller()
        elif choice == "2":
            find_travel_buddy()
        elif choice == "3":
            manage_todo()
        elif choice == "4":
            show_countdown()
        elif choice == "5":
            show_feedbacks()
        elif choice == "6":
            compare_views_vs_registrations()
        elif choice == "7":
            print("Exiting... Have a great day!")
            break
        else:
            print("Invalid choice. Please try again.")

if __name__ == "__main__":
    conn = sqlite3.connect("travel_app.db")
    cur = conn.cursor()
    cur.execute("""
        CREATE TABLE IF NOT EXISTS indian_travellers (
            name TEXT,
            destination TEXT,
            start_date TEXT,
            end_date TEXT,
            todo_list TEXT
        )
    """)
    cur.execute("""
        CREATE TABLE IF NOT EXISTS feedbacks (
            name TEXT,
            destination TEXT,
            feedback TEXT
        )
    """)
    conn.commit()
    conn.close()

    extended_main()



Travel Inspiration
'Not all those who wander are lost.' – J.R.R. Tolkien
----------------------------------------

--- Travel Planner App ---
1. Register Traveller
2. Find Travel Buddy
3. Manage To-Do List
4. Show Countdown
5. Show Feedbacks
6. Compare Views vs Registrations
7. Exit


Choose an option:  2



--- Find a Travel Buddy ---


Enter the destination you are traveling to:  Mumbai



Travellers going to Mumbai:
- mf (From 2024-01-21 to 2024-03-21)
- manas (From 2024-01-22 to 2024-03-22)
- manas (From 2024-03-31 to 2024-05-31)
- ASS (From 2025-09-08 to 2025-10-06)
- Manasvi Bhardwaj (From 2025-08-01 to 2025-08-12)
- Manasvi Bhardwaj (From 2025-08-01 to 2025-09-01)
- Manasvi Bhardwaj (From 2025-05-01 to 2025-06-01)
- Manasvi (From 2025-08-09 to 2025-08-18)

Travel Inspiration
'Life is short and the world is wide.' – Simon Raven
----------------------------------------

--- Travel Planner App ---
1. Register Traveller
2. Find Travel Buddy
3. Manage To-Do List
4. Show Countdown
5. Show Feedbacks
6. Compare Views vs Registrations
7. Exit


Choose an option:  7


Exiting... Have a great day!
